## Урок 9. Кейс 2. Внедрение модели в продукцию

In [1]:
import pandas as pd

import logging
import traceback
from logging.handlers import RotatingFileHandler
from time import strftime

from flask import Flask, request, jsonify
import datetime
from datetime import datetime

import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,20 hours 48 mins
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,5 months and 20 days !!!
H2O cluster name:,H2O_from_python_alxz_e1zf9o
H2O cluster total nodes:,1
H2O cluster free memory:,1.973 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [3]:
model_glm_poisson = h2o.load_model('E:\GB\MachineLearning\GLM_model_python_1591288035084_1')
model_glm_gamma = h2o.load_model('E:\GB\MachineLearning\GLM_model_python_1591288035084_2')

In [4]:
def map_for_dict_Gender(Gender):
    dict_Gender = {'Male':0, 'Female':1}
    res = dict_Gender.get(Gender)
    return res

def map_for_dict_MariStat(MariStat):
    dict_MariStat = {'Other':0, 'Alone':1}
    res = dict_MariStat.get(MariStat)
    return res

def f_VehUsage_Professional(VehUsage):
    if VehUsage == 'Professional':
        VehUsage_Professional = 1
    else:
        VehUsage_Professional = 0
    return(VehUsage_Professional)

def f_VehUsage_Private_trip_to_office(VehUsage):
    if VehUsage == 'Private+trip to office':
        VehUsage_Private_trip_to_office = 1
    else:
        VehUsage_Private_trip_to_office = 0
    return(VehUsage_Private_trip_to_office)

def f_VehUsage_Private(VehUsage):
    if VehUsage == 'Private':
        VehUsage_Private = 1
    else:
        VehUsage_Private = 0
    return(VehUsage_Private)

def f_VehUsage_Professional_run(VehUsage):
    if VehUsage == 'Professional run':
        VehUsage_Professional_run = 1
    else:
        VehUsage_Professional_run = 0
    return(VehUsage_Professional_run)

def return_NewH2o_Frame():
    columns = [
      'LicAge',
      'Gender',
      'MariStat',
      'DrivAge',
      'HasKmLimit',
      'BonusMalus',
      'OutUseNb',
      'RiskArea',
      'VehUsg_Private',
      'VehUsg_Private+trip to office',
      'VehUsg_Professional',
      'VehUsg_Professional run',
      'CSP1',
      'CSP2',
      'CSP3',
      'CSP6',
      'CSP7',
      'CSP20',
      'CSP21',
      'CSP22',
      'CSP26',
      'CSP37',
      'CSP40',
      'CSP42',
      'CSP46',
      'CSP47',
      'CSP48',
      'CSP49',
      'CSP50',
      'CSP55',
      'CSP56',
      'CSP57',
      'CSP60',
      'CSP65',
      'CSP66'
      ]

    return h2o.H2OFrame([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
                        column_names=columns)

In [5]:
handler = RotatingFileHandler('app.log', maxBytes = 100000, backupCount = 5)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(handler)

In [6]:
app = Flask(__name__)

@app.route('/')
def index():
    return 'API 4 predict srvc'

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    json_input = request.json
    
    current_datatime = strftime('%Y-%b-%d %H:%M:%S')
    ip_address = request.headers.get('X-forwarded-For', request.remote_addr)
    logger.info(f'{current_datatime} request from {ip_address}: {request.json}')
    start_prediction = datetime.now()

    ID = json_input["ID"]
    LicAge = json_input["LicAge"]
    Gender = map_for_dict_Gender(json_input["Gender"])
    MariStat = map_for_dict_MariStat(json_input["MariStat"])
    DrivAge = json_input["DrivAge"]
    HasKmLimit = json_input["HasKmLimit"]
    BonusMalus = json_input["BonusMalus"]
    OutUseNb = json_input["OutUseNb"]
    RiskArea = json_input["RiskArea"]
    VehUsg_Private = f_VehUsage_Private(json_input["VehUsage"])
    VehUsg_Private_trip_to_office = f_VehUsage_Private_trip_to_office(json_input["VehUsage"])
    VehUsg_Professional = f_VehUsage_Professional(json_input["VehUsage"])
    VehUsg_Professional_run = f_VehUsage_Professional_run(json_input["VehUsage"])
    CSP1 = 0
    CSP2 = 0
    CSP3 = 0
    CSP4 = 0
    CSP5 = 0
    CSP6 = 0
    CSP7 = 0
    CSP20 = 0
    CSP21 = 0
    CSP22 = 0
    CSP26 = 0
    CSP37 = 0
    CSP40 = 0
    CSP42 = 0
    CSP46 = 0
    CSP47 = 0
    CSP48 = 0
    CSP49 = 0
    CSP50 = 0
    CSP55 = 0
    CSP56 = 0
    CSP57 = 0
    CSP60 = 0
    CSP65 = 0
    CSP66 = 0

    hf = return_NewH2o_Frame()

    hf[0, 'LicAge'] = LicAge
    hf[0, 'Gender'] = Gender
    hf[0, 'MariStat'] = MariStat
    hf[0, 'DrivAge'] = DrivAge
    hf[0, 'HasKmLimit'] = HasKmLimit
    hf[0, 'BonusMalus'] = BonusMalus
    hf[0, 'OutUseNb'] = OutUseNb
    hf[0, 'RiskArea'] = RiskArea
    hf[0, 'VehUsg_Private'] = VehUsg_Private
    hf[0, 'VehUsg_Private+trip to office'] = VehUsg_Private_trip_to_office
    hf[0, 'VehUsg_Professional'] = VehUsg_Professional
    hf[0, 'VehUsg_Professional run'] = VehUsg_Professional_run
    hf[0, 'DrivAgeSq'] = DrivAge ** 2
    hf[0, 'CSP1'] = CSP1
    hf[0, 'CSP2'] = CSP2
    hf[0, 'CSP3'] = CSP3
    hf[0, 'CSP4'] = CSP4
    hf[0, 'CSP5'] = CSP5
    hf[0, 'CSP6'] = CSP6
    hf[0, 'CSP7'] = CSP7
    hf[0, 'CSP20'] = CSP20
    hf[0, 'CSP21'] = CSP21
    hf[0, 'CSP22'] = CSP22
    hf[0, 'CSP26'] = CSP26
    hf[0, 'CSP37'] = CSP37
    hf[0, 'CSP40'] = CSP40
    hf[0, 'CSP42'] = CSP42
    hf[0, 'CSP46'] = CSP46
    hf[0, 'CSP47'] = CSP47
    hf[0, 'CSP48'] = CSP48
    hf[0, 'CSP49'] = CSP49
    hf[0, 'CSP50'] = CSP50
    hf[0, 'CSP55'] = CSP55
    hf[0, 'CSP56'] = CSP56
    hf[0, 'CSP57'] = CSP57
    hf[0, 'CSP60'] = CSP60
    hf[0, 'CSP65'] = CSP65
    hf[0, 'CSP66'] = CSP66

    prediction_Poisson = model_glm_poisson.predict(hf)
    value_Poisson  = prediction_Poisson.as_data_frame()['predict'][0]
    prediction_Gamma = model_glm_gamma.predict(hf)
    value_Gamma  = prediction_Gamma.as_data_frame()['predict'][0]
    value_BurningCost = value_Poisson * value_Gamma

    result = {'ID': ID, 'value_Poisson': value_Poisson, 'value_Gamma': value_Gamma, 'value_BurningCost': value_BurningCost}
    
    end_prediction = datetime.now()
    duration = end_prediction-start_prediction
    current_dataframe = strftime('[%Y-%b-%d %H:%M:%S]')
    logger.info(f'{current_datatime} predicted for {duration} msec: {result}\n')
    
    result = jsonify(result)
    
    return result

In [7]:
@app.errorhandler(Exception)
def exceptions(e):
    current_datatime = strftime('%Y-%b-%d %H:%M:%S')
    error_message = traceback.format_exc()
    logger.error('%s %s %s %s %s 5xx Internal Server Error\n%s',
                 current_datatime,
                 request.remote_addr,
                 request.method,
                 request.scheme,
                 request.full_path,
                 error_message)
                 
    return jsonify({'error': 'Internal Server Error'}), 500

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:__main__:2020-Jun-05 20:50:58 request from 127.0.0.1: {'ID': 5, 'LicAge': 455, 'RecordBeg': '2004-01-01', 'RecordEnd': '', 'VehAge': 100, 'Gender': 'Male', 'MariStat': 'Alone', 'SocioCateg': 'CSP50', 'VehUsage': 'Private+trip to office', 'DrivAge': 45, 'HasKmLimit': 0, 'BonusMalus': 50, 'OutUseNb': 0, 'RiskArea': 3}


Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


INFO:__main__:2020-Jun-05 20:50:58 predicted for 0:00:28.673663 msec: {'ID': 5, 'value_Poisson': 0.2236396573747494, 'value_Gamma': 845.1394204746242, 'value_BurningCost': 189.00669042883922}

INFO:werkzeug:127.0.0.1 - - [05/Jun/2020 20:51:29] "POST /predict HTTP/1.1" 200 -
